# Training a neural network in PyTorch
This notebook demonstrates training a classifier in PyTorch.

In [3]:
import zarr
import os
import dask
import dask.array 
import torch
import numpy as np
import pathlib

In [5]:
# load in the data

def load_data_from_zarr(path):
    
    store = zarr.DirectoryStore(training_data_path)
    zarr_group = zarr.group(store=store)
    print("Loaded zarr, file information:\n", zarr_group.info, "\n")
    
    x = dask.array.from_zarr(zarr_group['humidity_temp_pressure_x.zarr'])
    y_lab = dask.array.from_zarr(zarr_group['onehot_cloud_base_height_y.zarr'])
    y_cont = dask.array.from_zarr(zarr_group['cloud_volume_fraction_y.zarr'])
    
    return x, y_lab, y_cont

root_data_directory = pathlib.Path(os.environ['SCRATCH']) / 'cbh_data'

dev_data_path = root_data_directory / 'analysis_ready' / 'dev.zarr' 
training_data_path = root_data_directory / 'analysis_ready' / 'train.zarr'

train_input, train_labels, train_cloud_volume = load_data_from_zarr(training_data_path)
dev_input, dev_labels, dev_cloud_volume = load_data_from_zarr(dev_data_path)

Loaded zarr, file information:
 Name        : /
Type        : zarr.hierarchy.Group
Read-only   : False
Store type  : zarr.storage.DirectoryStore
No. members : 3
No. arrays  : 3
No. groups  : 0
Arrays      : cloud_volume_fraction_y.zarr, humidity_temp_pressure_x.zarr,
            : onehot_cloud_base_height_y.zarr
 

Loaded zarr, file information:
 Name        : /
Type        : zarr.hierarchy.Group
Read-only   : False
Store type  : zarr.storage.DirectoryStore
No. members : 3
No. arrays  : 3
No. groups  : 0
Arrays      : cloud_volume_fraction_y.zarr, humidity_temp_pressure_x.zarr,
            : onehot_cloud_base_height_y.zarr
 



## Define the network

In [9]:
# define RNN
class CloudBaseLSTM(torch.nn.Module):
    def __init__(self, inputSize, lstmLayers, lstmHiddenSize, output_size, height_dimension, embed_size, BILSTM=True, batch_first=False):
        super(CloudBaseLSTM, self).__init__()
        
        self.LSTM = torch.nn.LSTM(inputSize+embed_size, lstmHiddenSize, lstmLayers, batch_first=batch_first, bidirectional=BILSTM, proj_size=output_size)
        
        self.linearCap = torch.nn.Linear(height_dimension, height_dimension)
        
        self.batch_first = batch_first
        self.proj_size = output_size
        
        self.relu = torch.nn.ReLU()
        
        self.height_embedding = torch.nn.Embedding(height_dimension, embed_size)
        self.BILSTM = BILSTM
        
    def forward(self, x, height):
        
        #produce height embeds
        height_embeds = self.height_embedding(height)
        height_embeds = torch.flatten(height_embeds, start_dim=2)
        # print(height_embeds.size())
        
        #concat with feature vector
        x_and_height = torch.cat((x, height_embeds), 2)
        
        #send through LSTM
        lstm_out, _ = self.LSTM(x_and_height)
        # combine backward and forward LSTM outputs for each cell
        if(self.BILSTM):
            lstm_out = lstm_out[:,:,:self.proj_size] + lstm_out[:,:,self.proj_size:]
        # combinedLSTMOut = combinedLSTMOut / 2
        
        # # softmax but check for batch first
        # softmax_dim = 0
        # if self.batch_first:
        #     softmax_dim = 1
            
            
        # flatten seq out
        lstm_out = torch.flatten(lstm_out, start_dim=1)
        
        # #normalization
        # out = torch.nn.functional.log_softmax(nn_out, dim=softmax_dim)
        
        # apply ReLU
        relu_out = self.relu(lstm_out)
        
        # apply linear layer for base prediction
        nn_out = self.linearCap(relu_out)
        
        # return both the nn_out and the lstm out for loss calculations
        return nn_out, relu_out

# define torch dataloader
class CBH_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_x, data_y, cloud_base_label):
        print("begin init")
        
        self.temp_humidity_pressure = data_x
        self.cloudbase_target = data_y
        self.cbh_label = cloud_base_label
        
        self.height_layer_number = data_x.shape[1] # take the shape at index 1 as data_x of format sample, height, feature
        
        assert self.height_layer_number == 70
        
        # legacy cloud base height
        # self.height_encoding = torch.from_numpy(data_height)
        print("end init")
        
    def __len__(self):
        return len(self.temp_humidity_pressure)

    def __getitem__(self, idx):
        
        # since dask is being used, first compute the values on the index given to the get function, convert the array to tensor for pytorch
        
        input_features = torch.from_numpy(self.temp_humidity_pressure[idx].compute())
        output_target = torch.from_numpy(self.cloudbase_target[idx].compute())
        output_target = output_target.type(torch.FloatTensor)
        cbh_lab = torch.from_numpy(self.cbh_label[idx].compute())
        
        print("CALL ON GETITEM")
        
        height_vec = torch.from_numpy(np.arange(self.height_layer_number)) # should have produced this vector here, as it is the same every time, but will leave it since sunken cost and maybe it improves performance??? 
        
        item_in_dataset = {'x':input_features, 'cloud_volume_target':output_target, 'cloud_base_target':cbh_lab, 'height_vector':height_vec}
        return item_in_dataset
    
    
# define Training loop
def trainCloudBaseLSTM(epochs, model, optimizer, cloud_volume_loss, cloud_base_loss, train_dataset, dev_dataset, batch_size, report_after_step, collate_fn=None):
    # TODO: integrate usage of dev dataset while training
    
    collate_fn=collate_fn
    # create dataloader
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    for epoch in range(epochs):
        for step, sample in enumerate(train_dataloader):
            # Compute prediction and loss
            base_pred, vol_pred = model(sample['x'], sample['height_vector'])
            loss_1 = cloud_volume_loss(vol_pred, sample['cloud_volume_target'])
            loss_2 = cloud_base_loss(base_pred, sample['cloud_base_target'])
            loss = (loss_1*40) + loss_2

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # report on progress
            if step%report_after_step == 0:
                print("epoch:", epoch, "step:", step)
                print("training base loss",loss_2,"/// training volume loss",loss_1,'\n')
    
    print("Training, end. Final loss:", loss_2)

In [12]:
# define dask specific collate function for dataloader, collate is the step where the dataloader combines all the samples into a singular batch to be enumerated on, 
# after getting all items 

def dataloader_collate_with_dask(batch):
    
    assert torch.utils.data.get_worker_info() is None # if this assertion fails, there are issues in code and this case needs to be handled see pytorch source of default collate fn
    print(batch)
    return 1
    
    elem = batch[0]
    elem_type = type(elem)
    if isinstance(elem, torch.Tensor):
        out = None
        if torch.utils.data.get_worker_info() is not None:
            # If we're in a background process, concatenate directly into a
            # shared memory tensor to avoid an extra copy
            numel = sum(x.numel() for x in batch)
            storage = elem.storage()._new_shared(numel, device=elem.device)
            out = elem.new(storage).resize_(len(batch), *list(elem.size()))
        return torch.stack(batch, 0, out=out)
    elif elem_type.__module__ == 'numpy' and elem_type.__name__ != 'str_' \
            and elem_type.__name__ != 'string_':
        if elem_type.__name__ == 'ndarray' or elem_type.__name__ == 'memmap':
            # array of string classes and object
            if np_str_obj_array_pattern.search(elem.dtype.str) is not None:
                raise TypeError(default_collate_err_msg_format.format(elem.dtype))

            return default_collate([torch.as_tensor(b) for b in batch])
        elif elem.shape == ():  # scalars
            return torch.as_tensor(batch)
    elif isinstance(elem, collections.abc.Mapping):
        try:
            return elem_type({key: default_collate([d[key] for d in batch]) for key in elem})
        except TypeError:
            # The mapping type may not support `__init__(iterable)`.
            return {key: default_collate([d[key] for d in batch]) for key in elem}
    elif isinstance(elem, collections.abc.Sequence):
        # check to make sure that the elements in batch have consistent size
        it = iter(batch)
        elem_size = len(next(it))
        if not all(len(elem) == elem_size for elem in it):
            raise RuntimeError('each element in list of batch should be of equal size')
        transposed = list(zip(*batch))  # It may be accessed twice, so we use a list.

        if isinstance(elem, tuple):
            return [default_collate(samples) for samples in transposed]  # Backwards compatibility.
        else:
            try:
                return elem_type([default_collate(samples) for samples in transposed])
            except TypeError:
                # The sequence type may not support `__init__(iterable)` (e.g., `range`).
                return [default_collate(samples) for samples in transposed]

    raise TypeError(default_collate_err_msg_format.format(elem_type))
    

## Perform the network initialization and training

In [13]:
# split dev and train


# load into dataset
train_input, train_cloud_volume, train_labels

collate_fn = dataloader_collate_with_dask

train_cbh_data = CBH_Dataset(train_input, train_cloud_volume, train_labels)
dev_cbh_data = CBH_Dataset(dev_input, dev_cloud_volume, dev_labels)

height_dim = train_input.shape[1]

# define model and hyperparameters
layers = 3
# input size is the cell input (feat dim)
input_size = train_input.shape[2]
output_size = 1 # for each height layer, predict one value for cloud base prob
hidden_size = 32
embed_size = 5
BILSTM = False
batch_first = True

learn_rate = 0.002

model = CloudBaseLSTM(input_size, layers, hidden_size, output_size, height_dim, embed_size, BILSTM, batch_first)
optim = torch.optim.Adam(model.parameters(), learn_rate)
loss_fn_vol = torch.nn.MSELoss()
loss_fn_base = torch.nn.CrossEntropyLoss()
epochs = 10
batch_size = 10

report_after_train_step = 100

# train
print("Started train")
trainCloudBaseLSTM(epochs, model, optim, loss_fn_vol, loss_fn_base, train_cbh_data, dev_cbh_data, batch_size, report_after_train_step, collate_fn)

begin init
end init
begin init
end init
Started train
CALL ON GETITEM
CALL ON GETITEM
CALL ON GETITEM
CALL ON GETITEM
CALL ON GETITEM
CALL ON GETITEM
CALL ON GETITEM
CALL ON GETITEM
CALL ON GETITEM
CALL ON GETITEM
OKBEGIN
None
OK
[{'x': tensor([[9.3454e-01, 7.9427e-01, 3.1811e-01],
        [9.3078e-01, 7.9140e-01, 3.1706e-01],
        [9.2555e-01, 7.8853e-01, 3.1560e-01],
        [9.1884e-01, 7.8566e-01, 3.1359e-01],
        [9.1070e-01, 7.8136e-01, 3.1381e-01],
        [9.0114e-01, 7.7634e-01, 3.1186e-01],
        [8.9019e-01, 7.7133e-01, 3.1206e-01],
        [8.7791e-01, 7.6487e-01, 3.1346e-01],
        [8.6432e-01, 7.5771e-01, 3.1305e-01],
        [8.4948e-01, 7.5054e-01, 3.0861e-01],
        [8.3345e-01, 7.4337e-01, 3.0124e-01],
        [8.1629e-01, 7.3763e-01, 2.8656e-01],
        [7.9808e-01, 7.3118e-01, 2.7271e-01],
        [7.7888e-01, 7.2330e-01, 2.5696e-01],
        [7.5877e-01, 7.1398e-01, 2.4170e-01],
        [7.3779e-01, 7.0466e-01, 2.2412e-01],
        [7.1603e-01, 6.9391

TypeError: 'int' object is not subscriptable

## Display and evaluate results

In [ ]:
# sample some predictions
#have a look at model predictions for a sample after training
# {'x':input_features, 'cloud_volume_target':output_target, 'cloud_base_target':cbh_lab, 'height_vector':height_vec}

sample = cbh_data[1:2]

modelOutBase, moVol = model(sample['x'], sample['height_vector'])
print(modelOutBase)
print(sample['cloud_base_target'])
print('')
print(moVol)
print(sample['cloud_volume_target'])


# predictionInit = torch.zeros(modelOut.size(1))
# predictionInit[torch.argmax(modelOut,dim=1)] = 1
# print(predictionInit)